# Language-Recognition using ConvNets

_written by Joscha S. Rieber (Fraunhofer IAIS) in 2020_

## Training Environment

In [1]:
train = 'train'
test = 'test'

eng = 'english'
ger = 'german'

languages = [eng, ger]
categories = [train, test]

dataset_root_path = '../data/'
train_path = dataset_root_path + train

batch_size = 128
image_width = 500
image_height = 128

validation_split = 0.1
initial_learning_rate = 0.045

num_classes = len(languages)

model_file = dataset_root_path + 'model.h5'

In [2]:
from glob import glob
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# XLA compiles your TensorFlow graph into a sequence of GPU kernels generated specifically for your model.
# Since these kernels are unique to your program, they can exploit model-specific information for optimization.

# import os
# os.environ['TF_XLA_FLAGS'] = '--tf_xla_cpu_global_jit'

In [3]:
all_files = glob(train_path + '/*/*.png')

num_validation = len(all_files) * validation_split
num_train = len(all_files) - num_validation

validation_steps = int(num_validation / batch_size)
steps_per_epoch = int(num_train / batch_size)

print('Steps per Epoch: ' + str(steps_per_epoch))
print('Validation steps: ' + str(validation_steps))

Steps per Epoch: 450
Validation steps: 50


## Training Batch Generator Function

The following function loads the available images for training, shuffles them and serves them to Keras' training algorithm.

In [4]:
image_data_generator = ImageDataGenerator(rescale=1./255, validation_split=validation_split)
train_generator = image_data_generator.flow_from_directory(train_path, batch_size=batch_size, class_mode='categorical', target_size=(image_height, image_width), color_mode='grayscale', subset='training')
validation_generator = image_data_generator.flow_from_directory(train_path, batch_size=batch_size, class_mode='categorical', target_size=(image_height, image_width), color_mode='grayscale', subset='validation')

Found 57600 images belonging to 2 classes.
Found 6400 images belonging to 2 classes.


## Model definition

In [5]:
import math

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, AveragePooling2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Input, Concatenate, Activation
from tensorflow.keras import regularizers
from tensorflow.keras.optimizers import SGD, RMSprop

In [6]:
from tensorflow.python.keras.layers import Concatenate
from tensorflow.python.keras.applications.inception_v3 import InceptionV3

img_input = Input(shape=(image_height, image_width, 1))

img_conc = Concatenate(axis=3, name='input_concat')([img_input, img_input, img_input])

model = InceptionV3(input_tensor=img_conc, weights=None, include_top=True, classes=2)

model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 500, 1) 0                                            
__________________________________________________________________________________________________
input_concat (Concatenate)      (None, 128, 500, 3)  0           input_1[0][0]                    
                                                                 input_1[0][0]                    
                                                                 input_1[0][0]                    
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 63, 249, 32)  864         input_concat[0][0]               
_______________________________________________________________________________________

In [7]:
#model = Sequential()

#model.add(Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=(image_height, image_width, 1)))
#model.add(MaxPooling2D())
#model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
#model.add(MaxPooling2D())
#model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
#model.add(MaxPooling2D())
#model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
#model.add(MaxPooling2D())
#model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
#model.add(MaxPooling2D())
#model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(num_classes, activation='softmax'))

#model.summary()

In [8]:
model.compile(optimizer=RMSprop(lr=initial_learning_rate, clipvalue=2.0), loss='categorical_crossentropy', metrics=['accuracy'])

## Training

In [9]:
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler

In [10]:
early_stopping = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)

In [11]:
def step_decay(epoch, lr):
    drop = 0.94
    epochs_drop = 2.0
    lrate = lr * math.pow(drop, math.floor((1+epoch)/epochs_drop))
    return lrate

learning_rate_decay = LearningRateScheduler(step_decay, verbose=1)

In [12]:
model.fit(train_generator, validation_data=validation_generator, epochs=60, steps_per_epoch=steps_per_epoch, validation_steps=validation_steps, callbacks=[early_stopping, learning_rate_decay])


Epoch 00001: LearningRateScheduler reducing learning rate to 0.04500000178813934.
Epoch 1/60
450/450 [==============================] - 438s 974ms/step - loss: 1.0906 - accuracy: 0.5219 - val_loss: 928.9180 - val_accuracy: 0.5025 - lr: 0.0450

Epoch 00002: LearningRateScheduler reducing learning rate to 0.042300001680850983.
Epoch 2/60
450/450 [==============================] - 436s 969ms/step - loss: 0.4956 - accuracy: 0.7728 - val_loss: 8.0367 - val_accuracy: 0.5213 - lr: 0.0423

Epoch 00003: LearningRateScheduler reducing learning rate to 0.03976200073957443.
Epoch 3/60
450/450 [==============================] - 434s 965ms/step - loss: 0.3053 - accuracy: 0.8706 - val_loss: 0.6972 - val_accuracy: 0.8103 - lr: 0.0398

Epoch 00004: LearningRateScheduler reducing learning rate to 0.03513370451182127.
Epoch 4/60
450/450 [==============================] - 433s 963ms/step - loss: 0.2320 - accuracy: 0.9053 - val_loss: 0.4877 - val_accuracy: 0.7897 - lr: 0.0351

Epoch 00005: LearningRateSch

In [13]:
model.save(model_file)

## Model Evaluation Environment


The following function loads the available images for evaluation

In [14]:
test_path = dataset_root_path + test

In [15]:
all_files = glob(test_path + '/*/*.png')

num_test = len(all_files)

evaluation_steps = int(num_test / batch_size)

print('Evaluation steps: ' + str(evaluation_steps))

Evaluation steps: 62


In [16]:
image_data_generator = ImageDataGenerator(rescale=1./255)
evaluation_generator = image_data_generator.flow_from_directory(test_path, batch_size=batch_size, class_mode='categorical', target_size=(image_height, image_width), color_mode='grayscale')

Found 8000 images belonging to 2 classes.


## Model evaluation

In [17]:
from tensorflow.keras.models import load_model

In [18]:
model = load_model(model_file)

In [19]:
_, test_accuracy = model.evaluate(evaluation_generator, steps=evaluation_steps)

62/62 [==============================] - 33s 526ms/step - loss: 0.6145 - accuracy: 0.9403


In [20]:
print('Test accuracy: ' + str(round(test_accuracy * 100., 1)) + ' %')

Test accuracy: 94.0 %
